# ChIP-seq Analysis Report

This notebook contains all figures, visualizations, and written discussion for the ChIP-seq analysis pipeline.

## Project Overview

This analysis processes single-end ChIP-seq data to:
1. Perform quality control and read alignment
2. Generate coverage tracks and assess sample correlation
3. Call peaks using HOMER
4. Identify reproducible peaks and filter blacklist regions
5. Annotate peaks to genomic features
6. Visualize signal across gene bodies
7. Identify enriched motifs in peaks

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path
from IPython.display import Image, display

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 100

## 1. Quality Control Summary

### MultiQC Report

The MultiQC report aggregates quality control metrics from FastQC, Trimmomatic, Bowtie2 alignment, and samtools flagstat.

**Key metrics to review:**
- Raw read quality scores
- Adapter content
- Trimming statistics
- Alignment rates
- Duplicate rates

In [ ]:
# Display MultiQC report (if available)
multiqc_path = Path('results/multiqc/multiqc_report.html')
if multiqc_path.exists():
    print(f"MultiQC report available at: {multiqc_path}")
    print("Open this HTML file in a browser to view comprehensive QC metrics.")
else:
    print("MultiQC report not yet generated. Run the pipeline first.")

## 2. Sample Correlation Analysis

### Correlation Heatmap

The correlation heatmap shows the similarity between samples based on their bigWig coverage profiles.

**Expected patterns:**
- IP samples (IP_rep1, IP_rep2) should cluster together with high correlation
- INPUT samples (INPUT_rep1, INPUT_rep2) should cluster together
- IP and INPUT samples should show lower correlation

**Correlation method:** Spearman correlation was chosen because:
- More robust to outliers than Pearson correlation
- Better suited for non-normal distributions typical in ChIP-seq data
- Captures monotonic relationships rather than strictly linear ones

In [ ]:
# Display correlation plot
corr_plot = Path('results/plotcorrelation/correlation_plot.png')
if corr_plot.exists():
    display(Image(filename=str(corr_plot)))
    print("\nCorrelation heatmap showing sample similarity based on bigWig profiles.")
else:
    print("Correlation plot not yet generated. Run the pipeline first.")

In [ ]:
# Load and display correlation matrix values
corr_matrix_path = Path('results/plotcorrelation/correlation_matrix.tab')
if corr_matrix_path.exists():
    corr_df = pd.read_csv(corr_matrix_path, sep='\t', index_col=0)
    print("Correlation Matrix:")
    print(corr_df)
    print("\nInterpretation:")
    print(f"- IP samples correlation: {corr_df.loc['IP_rep1', 'IP_rep2']:.3f}")
    print(f"- INPUT samples correlation: {corr_df.loc['INPUT_rep1', 'INPUT_rep2']:.3f}")
else:
    print("Correlation matrix not yet generated.")

## 3. Peak Calling Results

### Peak Statistics

HOMER was used to call peaks with the following parameters:
- Style: factor (for transcription factor binding)
- Control: INPUT samples used as background
- Replicates: Two biological replicates processed independently

In [ ]:
# Load peak statistics
def count_peaks(bed_file):
    """Count number of peaks in a BED file"""
    if Path(bed_file).exists():
        with open(bed_file) as f:
            return sum(1 for line in f if not line.startswith('#'))
    return 0

# Count peaks at each stage
rep1_peaks = count_peaks('results/homer_pos2bed/rep1_peaks.bed')
rep2_peaks = count_peaks('results/homer_pos2bed/rep2_peaks.bed')
repr_peaks = count_peaks('results/bedtools_intersect/repr_peaks.bed')
filtered_peaks = count_peaks('results/bedtools_remove/repr_peaks_filtered.bed')

print("Peak Calling Summary:")
print(f"  Replicate 1 peaks: {rep1_peaks:,}")
print(f"  Replicate 2 peaks: {rep2_peaks:,}")
print(f"  Reproducible peaks (intersection): {repr_peaks:,}")
print(f"  After blacklist filtering: {filtered_peaks:,}")
print(f"\n  Peaks removed by blacklist: {repr_peaks - filtered_peaks:,} ({100*(repr_peaks - filtered_peaks)/repr_peaks:.1f}%)")

In [ ]:
# Visualize peak overlap
peak_data = {
    'Stage': ['Rep1', 'Rep2', 'Reproducible', 'Filtered'],
    'Count': [rep1_peaks, rep2_peaks, repr_peaks, filtered_peaks]
}

if rep1_peaks > 0 or rep2_peaks > 0:
    fig, ax = plt.subplots(figsize=(8, 5))
    bars = ax.bar(peak_data['Stage'], peak_data['Count'], color=['skyblue', 'lightcoral', 'lightgreen', 'orange'])
    ax.set_ylabel('Number of Peaks')
    ax.set_title('Peak Calling Results at Each Stage')
    ax.set_ylim(0, max(peak_data['Count']) * 1.1)
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height):,}',
                ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
else:
    print("Peak calling results not yet available.")

### Reproducible Peak Strategy

**Method:** We used `bedtools intersect -wa` to identify reproducible peaks.

**Rationale:**
- This approach identifies peaks that are present in both biological replicates
- Any overlap between replicate peaks is considered evidence of reproducibility
- The `-wa` flag reports the coordinates from the first replicate for peaks that overlap with the second replicate
- This is a conservative approach that ensures only peaks with supporting evidence from both replicates are retained

**Alternative strategies considered:**
- Requiring minimum overlap percentage (more stringent but may miss real peaks)
- Using IDR (Irreproducible Discovery Rate) framework (more sophisticated but computationally intensive)
- Simple union of peaks (less stringent, higher false positive rate)

## 4. Peak Annotation

### Genomic Distribution of Peaks

Peaks were annotated using HOMER's `annotatePeaks.pl` with custom genome and GTF files.

In [ ]:
# Load and analyze peak annotations
annot_file = Path('results/homer_annotate/annotated_peaks.txt')
if annot_file.exists():
    # Read HOMER annotation file (tab-separated)
    annot_df = pd.read_csv(annot_file, sep='\t', comment='#')
    print(f"Total annotated peaks: {len(annot_df):,}")
    print("\nFirst few annotated peaks:")
    print(annot_df.head())
    
    # Analyze annotation distribution
    if 'Annotation' in annot_df.columns:
        print("\nAnnotation Distribution:")
        print(annot_df['Annotation'].value_counts().head(10))
else:
    print("Peak annotations not yet generated.")

## 5. Gene Signal Profiles

### Signal Across Gene Bodies

Using deepTools, we visualized the average ChIP-seq signal across all genes in the genome.

**Method:**
- Gene coordinates from UCSC hg38 reference (TSS to TTS)
- All genes scaled to uniform size
- 2kb flanking regions added upstream and downstream
- Only IP samples shown (INPUT represents random background)

**Interpretation:**
- Peaks at TSS suggest promoter binding
- Signal throughout gene body suggests transcription-related binding
- Downstream signal may indicate terminator or 3'UTR binding

In [ ]:
# Display gene signal plots for IP samples
ip_samples = ['IP_rep1', 'IP_rep2']

for sample in ip_samples:
    plot_path = Path(f'results/plotprofile/{sample}_signal_coverage.png')
    if plot_path.exists():
        print(f"\n### {sample} Signal Profile")
        display(Image(filename=str(plot_path)))
    else:
        print(f"Signal profile for {sample} not yet generated.")

## 6. Motif Enrichment Analysis

### Enriched Motifs in Peaks

HOMER's `findMotifsGenome.pl` was used to identify enriched sequence motifs in reproducible, filtered peaks.

**Parameters:**
- Size: 200bp regions centered on peaks
- Background: Random genomic regions matched for GC content

**Expected Results:**
- Known motifs for the transcription factor of interest
- Motifs for co-factors or interacting proteins
- De novo discovered motifs

In [ ]:
# List motif results
motif_dir = Path('results/homer_motifs/motifs')
if motif_dir.exists():
    print("Motif analysis output directory:")
    print(f"  {motif_dir}")
    print("\nKey files:")
    print("  - homerResults.html: Main results page with all enriched motifs")
    print("  - knownResults.html: Results for known motif databases")
    print("  - homerMotifs.all.motifs: All discovered motifs in HOMER format")
    
    # Check if HTML results exist
    html_result = motif_dir / 'homerResults.html'
    if html_result.exists():
        print(f"\nOpen {html_result} in a browser to view detailed motif results.")
else:
    print("Motif enrichment analysis not yet completed.")

## 7. Summary and Conclusions

### Pipeline Completion Checklist

- [ ] Quality control passed (MultiQC review)
- [ ] Sample correlation shows expected patterns
- [ ] Peaks called in both replicates
- [ ] Reproducible peaks identified
- [ ] Blacklist regions filtered
- [ ] Peaks annotated to genomic features
- [ ] Gene signal profiles generated
- [ ] Motif enrichment analysis completed

### Key Findings

*To be filled in after pipeline completion with specific results for your dataset.*

### Recommendations for Follow-up Analysis

1. **Functional enrichment**: Perform GO term or pathway analysis on genes near peaks
2. **Differential binding**: If comparing conditions, identify differentially bound regions
3. **Integration with expression data**: Correlate binding with gene expression changes
4. **Motif validation**: Validate top enriched motifs with independent experiments
5. **Peak visualization**: Generate genome browser tracks for specific regions of interest

## References

### Tools Used

- **Nextflow**: Di Tommaso et al., Nature Biotechnology 2017
- **FastQC**: Andrews S., Babraham Bioinformatics 2010
- **Trimmomatic**: Bolger et al., Bioinformatics 2014
- **Bowtie2**: Langmead and Salzberg, Nature Methods 2012
- **SAMtools**: Li et al., Bioinformatics 2009
- **deepTools**: Ramírez et al., Nucleic Acids Research 2016
- **HOMER**: Heinz et al., Molecular Cell 2010
- **BEDTools**: Quinlan and Hall, Bioinformatics 2010

### Resources

- **ENCODE Blacklist**: Amemiya et al., Scientific Reports 2019
- **UCSC Genome Browser**: Kent et al., Genome Research 2002